In [38]:
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sn
import geopandas as gpd
import copy
import typing 

In [4]:
!pip install xlrd

In [10]:
def load_data()-> dict:
    url = "https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-aq-1975-2023.xls"
    data = pd.read_excel(url,
    sheet_name=None,
    header=[3,4])
    return data

In [10]:
A = load_data()
A["2022"]

Départements  \
                                    Unnamed: 0_level_1   
0                                                   01   
1                                                   02   
2                                                   03   
3                                                   04   
4                                                   05   
..                                                 ...   
101                                                976   
102                                                DOM   
103                       France métropolitaine et DOM   
104                                                NaN   
105  Source : Insee - Estimations de population (ré...   

                               Ensemble                                     \
          Unnamed: 1_level_1  0 à 4 ans  5 à 9 ans 10 à 14 ans 15 à 19 ans   
0                        Ain    37584.0    44592.0     46872.0     41596.0   
1                      Aisne    26469.0    32510.0     35567.0     32782.0   
2                     Allier    13508.0    16754.0     18159.0     18286.0   
3    Alpes-de-Haute-Provence     7211.0     8751.0      9467.0      8656.0   
4               Hautes-Alpes     6041.0     7183.0      7943.0      7522.0   
..                       ...        ...        ...         ...         ...   
101                  Mayotte    47016.0    43895.0     39982.0     29873.0   
102                      NaN   172294.0   176252.0    182101.0    169777.0   
103                      NaN  3573900.0  4025667.0   4271387.0   4208524.0   
104                      NaN        NaN        NaN         NaN         NaN   
105                      NaN        NaN        NaN         NaN         NaN   

                                                     ...      Femmes  \
    20 à 24 ans 25 à 29 ans 30 à 34 ans 35 à 39 ans  ... 55 à 59 ans   
0       32455.0     32252.0     40494.0     44180.0  ...     22455.0   
1       28024.0     25419.0     29274.0     31434.0  ...     17976.0   
2       16279.0     14083.0     15296.0     16672.0  ...     11825.0   
3        6977.0      6822.0      8163.0      9022.0  ...      6528.0   
4        5521.0      5804.0      7219.0      7987.0  ...      5252.0   
..          ...         ...         ...         ...  ...         ...   
101     19284.0     19923.0     20467.0     19851.0  ...      3405.0   
102    127009.0    116519.0    129841.0    131092.0  ...     77860.0   
103   3922318.0   3725962.0   4064176.0   4200292.0  ...   2279864.0   
104         NaN         NaN         NaN         NaN  ...         NaN   
105         NaN         NaN         NaN         NaN  ...         NaN   

                                                                             \
    60 à 64 ans 65 à 69 ans 70 à 74 ans 75 à 79 ans 80 à 84 ans 85 à 89 ans   
0       20459.0     18543.0     17382.0     12387.0      8494.0      6821.0   
1       17975.0     17252.0     16532.0     10599.0      8058.0      6997.0   
2       12482.0     13125.0     13189.0      9667.0      7466.0      6559.0   
3        6420.0      6160.0      6044.0      4680.0      3413.0      2748.0   
4        5463.0      5291.0      5116.0      3683.0      2836.0      2189.0   
..          ...         ...         ...         ...         ...         ...   
101      2308.0      1570.0       981.0       701.0       375.0       216.0   
102     64319.0     55570.0     42452.0     30534.0     22192.0     14489.0   
103   2177958.0   2077890.0   1997938.0   1398856.0   1054963.0    870391.0   
104         NaN         NaN         NaN         NaN         NaN         NaN   
105         NaN         NaN         NaN         NaN         NaN         NaN   

                                            
    90 à 94 ans 95 ans et plus       Total  
0        3955.0         1583.0    337889.0  
1        3923.0         1290.0    268287.0  
2        3800.0         1410.0    173172.0  
3        1478.0          587.0     85859.0  
4        1207.0          375.0     71

In [69]:
def reshape_data_by_year(df, year):
    # Supp des NAN
    df = df.dropna().set_index("Départements")
    # Depivoter
    df = pd.melt(
        df.stack(level=0, future_stack=True).reset_index(),
        id_vars=["Départements", "level_1"],
        value_name="population",
        var_name="age"

    )

    # Renommage 
    df = df.rename(columns={
        "level_1": "genre"
        
    })
    # Spliting dep_code et dep
    df[["dep_code", "dep"]] = pd.DataFrame(df["Départements"].tolist())
    # Drop Departements
    df = df.drop(columns=["Départements"])
    # Processing 
    df["annee"] = year
    df["population"] = df["population"].astype("Int64")
    df["dep"] = df["dep"].astype(str)
    
    return df



In [64]:
data = load_data()

In [70]:

annee = 2022
df = reshape_data_by_year(data[f"{annee}"], annee)
df

,genre,age,population,dep_code,dep,annee
0,Ensemble,0 à 4 ans,37584,01,Ain,2022
1,Hommes,0 à 4 ans,19489,01,Ain,2022
2,Femmes,0 à 4 ans,18095,01,Ain,2022
3,Ensemble,0 à 4 ans,26469,02,Aisne,2022
4,Hommes,0 à 4 ans,13538,02,Aisne,2022
...,...,...,...,...,...,...
6358,Hommes,Total,413939,974,La Réunion,2022
6359,Femmes,Total,456054,974,La Réunion,2022
6360,Ensemble,Total,299022,976,Mayotte,2022
6361,Hommes,Total,142515,976,Mayotte,2022


In [ ]:
# Fonction qui produit un DataFrame avec les données pour toutes les années entre 1975 et 2022
def reshape_data(data):
    df = pd.concat(
        [reshape_data_by_year(data[f"{annee}"], annee) for year in range (1975, 2023)]
    )
    return df

Dataviz

In [ ]:
# Fonction qui renvoie unngraphique représentant l'évolution de la population dans un département donné

def plot_population_by_gender_per_department(df, department_code):
    return df


NameError: name 'df' is not defined

In [ ]:
# Création du DataFrame

def reshape_table_by_year(df, year):
    """
    This function reshapes a DataFrame by filtering and transforming it based on the year.

    Parameters:
        df (DataFrame): The input DataFrame to reshape.
        year (str): The year to filter the DataFrame by.

    Returns:
        DataFrame: The reshaped and filtered DataFrame.
    """
    # Removing NaNs and set Départements as Index
    df = df.dropna().set_index("Départements")

    # Reshaping dataframe by stacking levels and using melt function (from wide to long format)
    df = pd.melt(
        df.stack(level=0, future_stack=True).reset_index(),
        id_vars=["Départements", "level_1"],
        value_name="population",
        var_name="age",
    )

    # Renaming column
    df = df.rename(columns={"level_1": "genre"})

    # Splitting Départements column into dep_code and dep
    df[["dep_code", "dep"]] = pd.DataFrame(df["Départements"]).tolist()

    # Drop Départements column
    df = df.drop(columns=["Départements"])

    # Adding a year column
    df["annee"] = year

    # Making sure that population column is numeric
    df["population"] = df["population"].astype("int64")

    # Making sure that department code column is a string (problems with 0 and overseas territories)
    df["dep_code"] = df["dep_code"].astype(str)

    return df



In [ ]:
data = pd.read_excel(
        "https://www.insee.fr/fr/statistiques/fichier/1893198/estim-pop-dep-sexe-aq-1975-2023.xls",
        sheet_name=None, 
        header=[1,4]
    )
annee = 2022
df = reshape_table_by_year(data[f"{annee}"], annee)
df

KeyError: "None of ['Départements'] are in the columns"

In [ ]:
def reshape_data(data):
    df = pd.concat(
        [reshape_table_by_year(data[f"{year}"], year) for year in range(1975, 2023)]
    )
    return df